<a href="https://colab.research.google.com/github/anujgupta82/NLP_Bootcamp/blob/V_2_0/module4/Introduction%20to%20Character%20Embeddings%20(keras)_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Agenda - Given a sequence of previous characters, model the probability distribution of the next character in the sequence. 

Here we try harry potter text

We will be using kears for this note 

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls /content/drive/*

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.11-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.11-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.11-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

In [0]:
from __future__ import print_function

import numpy as np
import random
import sys

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

Using TensorFlow backend.


# Get data

In [4]:
# load data
path = "/content/drive/NLP_bootcamp/Data/harry_potter_3.txt"
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 611606


In [5]:
total_chars = sorted(list(set(text)))
print('total chars:', len(total_chars))


total chars: 53


In [0]:
#dictionaries to map characters to IDs and vice-a-versa

char_indices = dict((c, i) for i, c in enumerate(total_chars))
indices_char = dict((i, c) for i, c in enumerate(total_chars))


In [7]:
# cut the text in semi-redundant sequences of maxlen characters

maxlen = 40 # length of data window
step = 1    # step by which to shift the data window 

sentences = []    # this stores X
next_chars = []   # this stores y

# (i:i+40) as X, (i+40) as corresponding y
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen]) # get sentence
    next_chars.append(text[i + maxlen])   # get next character
    

num_of_sentences = len(sentences)

print('Number of datapoints/sequences:', num_of_sentences)

Number of datapoints/sequences: 611566


In [8]:
#Vectorize the input 

# X will be a 3D tensor - num_of_sentences * maxlen * total_chars
# y will be 2D tensor - sentences * total_chars

print('Vectorization...')
X = np.zeros((num_of_sentences, maxlen, len(total_chars)), dtype=np.bool)
y = np.zeros((num_of_sentences, len(total_chars)), dtype=np.bool)

# populate the tensors
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


# Model

In [9]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()

# lstm takes a single slice of 3D tensor along num_of_sentences axiss
model.add(LSTM(128, input_shape=(maxlen, len(total_chars)))) 

# add a dense layer - takes 128 input and spits output of num_of_chars 
model.add(Dense(len(total_chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...







In [0]:
#Preds is scores over output space (characters)
# Its nothing but - confidence the RNN currently assigns to each character coming next in the sequence

# we convert scores into probability distribution and pick the best bet. 

# Temperature. We can also play with the temperature of the Softmax during sampling. Decreasing the temperature 
# from 1 to some lower number (e.g. 0.5) makes the RNN more confident, but also more conservative in its samples. 
# Conversely, higher temperatures will give more diversity but at cost of more mistakes (e.g. spelling mistakes, 
# etc). In particular, setting temperature very near zero will give the most likely thing

def sample(preds, temperature=1.0):
    
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) # convert scores to prob via softmax
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas) # return the one with max probability

In [0]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1) # pick an index at random
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        
        generated = ''
        sentence = text[start_index: start_index + maxlen] # pick sentence at randomly gerenated index
        generated += sentence
        
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        sys.stdout.write("\n")
        
        for i in range(400):
            x = np.zeros((1, maxlen, len(total_chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1. # put the sentence in 1-hot format
                
            preds = model.predict(x, verbose=0)[0] # make the model spit out predictions
            
            next_index = sample(preds, diversity) # choose a character index
            next_char = indices_char[next_index]  # get the corresponding chracater
            
            generated += next_char                # append this character to the sentence
            sentence = sentence[1:] + next_char   # now shift the the sentence by one character
            
            sys.stdout.write(next_char)           # write the character to buffer
            sys.stdout.flush()
            
        print()


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1
611566/611566 [==============================] - 485s 793us/step - loss: 1.6931

----- diversity: 0.2
----- Generating with seed: "nagall,
aren't you, harry?"

"no, i'm no"
nagall,
aren't you, harry?"

"no, i'm no
t the grabbed the grip the stand and harry was a start the gray the start the stand the gringers and harry and hermione standing the cased the door and the door to have been black and hermione was a grinning at the disappeared the starting harry was a start the starting the dark arts and the dementors and the dementors and they was a start the door and the dementors was a talking the starting the 

----- diversity: 0.5
----- Generating with seed: "nagall,
aren't you, harry?"

"no, i'm no"
nagall,
aren't you, harry?"

"no, i'm no
t it was in the wand over the ward the light, harry was behind the saw that was al

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


isturie and the great and hermione down the staring and
wood toward
excelling in the wall. harry saw the face and the possible of the table, who was they saw anything table and they waid the before harry and hermione poened bat

----- diversity: 1.0
----- Generating with seed: "to ask for more time;
everyone else was "
to ask for more time;
everyone else was 
now
cloak pogching table, rolled "walk yelleal." harry said on socerhawry last across the dark,
looking.

"ron," said hermionely.

"shot," said harry.

"didn't mausivet -- de'n the throught's lin
w. filing, but you'd parned ; it as it, heard dear! does frelling grinnaticapum when the hoaches
the fhou. the gleasorib'
longwahograin.

"where has we slong to be stoppey that not air, it's comabs,.... h

----- diversity: 1.2
----- Generating with seed: "to ask for more time;
everyone else was "
to ask for more time;
everyone else was 
spinted the lapt tod quge
wiped would open naille garges, and harry and and the
ean's wrete,
hunding re